In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(
    train_file_path,
    encoding = "ISO-8859-1",
    sep="\t",
    names=['type', 'text'],
    usecols=['type', 'text'],
    dtype={'isbn': 'str', 'title': 'str'})
test_dataset = pd.read_csv(
    test_file_path,
    encoding = "ISO-8859-1",
    sep="\t",
    names=['type', 'text'],
    usecols=['type', 'text'],
    dtype={'isbn': 'str', 'title': 'str'})

train_labels = train_dataset.pop("type")
train_labels = (train_labels=="spam")*1.0
test_labels = test_dataset.pop("type")
test_labels = (test_labels=="spam")*1.0

train_dataset.head(10)

In [ ]:
max_features = 2**10
sequence_length = 2**5

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer.adapt(train_dataset)

train_ds = vectorize_layer(train_dataset)
test_ds = vectorize_layer(test_dataset)

train_ds[:10]

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(max_features, 16),
    keras.layers.LSTM(16),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_ds, train_labels, epochs=10, validation_data=(test_ds, test_labels))

In [ ]:
def encode_text(text):
  tokens = vectorize_layer(text)
  return tokens

text = "how are you doing today?"
encoded = encode_text(text)
print(encoded)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded_text = encode_text(pred_text)
  pred = np.zeros((1,sequence_length))
  pred[0] = encoded_text
  result = model.predict(pred)[0]
  return [result, "spam" if result>0.5 else "ham"]

pred_text = "how are you doing today?"
print(pred_text)
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
